# advent of code 2024 - [day 2](https://adventofcode.com/2024/day/2)

In [ ]:
import os

NEO4J_URI = os.environ['NEO4J_URI']
NEO4J_USERNAME = os.environ['NEO4J_USERNAME']
NEO4J_PASSWORD = os.environ['NEO4J_PASSWORD']


In [ ]:
from graphdatascience import GraphDataScience
gds = GraphDataScience(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# Check the installed GDS version on the server
print(gds.version())
assert gds.version()

from neo4j import GraphDatabase
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

import pandas as pd

In [ ]:
def infer (rules, params={}):
    """
    This is a function you can use if you want to run a set of inference rules
    until a convergence is reached. why not use it in a RDF-like reasoning context?
    """
    counter = 0
    while True:
        counter += 1
        any_update = False
        for rule in rules:
            with driver.session(database="neo4j") as session:
                result = session.run(rule, params)
            any_update = any_update or result.consume().counters._contains_updates
        if not any_update:
            break

In [ ]:
def gen_lists(file='input.txt'):
    """Generates tuples of integers"""
    file = open(file, 'r')
    for _, line in enumerate(file):
        els = line.strip().split(" ")
        yield list(int(el) for el in els)

## Python-based solution

### Part 1

In [ ]:
def is_safely_increasing(l):
    if len(l) < 2:
        return True
    return l[-2]+1 <= l[-1] <= l[-2]+3 and is_safely_increasing(l[:-1])


def is_safely_decreasing(l):
    return is_safely_increasing(list(reversed(l)))

def is_safely_monotonous(l):
    return is_safely_increasing(l) or is_safely_decreasing(l)

In [ ]:
monotonous_lists = (l for l in gen_lists() if is_safely_monotonous(l) )
part1 = len(list(monotonous_lists))
part1


### Part 2

In [ ]:
def is_safely_increasing(l):
    if len(l) < 2:
        return True
    return l[-2]+1 <= l[-1] <= l[-2]+3 and is_safely_increasing(l[:-1])

def is_safely_increasing_with_tolerance(l):
    return any(is_safely_increasing(l[:ix]+l[ix+1:]) for ix, _ in enumerate(l))

def is_safely_decreasing_with_tolerance(l):
    return is_safely_increasing_with_tolerance(list(reversed(l)))

def is_safely_monotonous_with_tolerance(l):
    return is_safely_increasing_with_tolerance(l) or is_safely_decreasing_with_tolerance(l)

In [ ]:
monotonous_lists = (l for l in gen_lists() if is_safely_monotonous_with_tolerance(l) )
part2 = len(list(monotonous_lists))
part2

## Neo4j-based solution

### Parsing

In [ ]:
reports = list(gen_lists())

### Ingestion

In [ ]:
query_ingest = """
UNWIND range(0, size($reports)-1) AS rep_ix
UNWIND range(0, size($reports[rep_ix])-1) AS rec_ix
WITH rep_ix AS report_id, rec_ix AS record_number, $reports[rep_ix][rec_ix] AS value
CREATE (:Record {report_id: report_id, record_number: record_number, value: value})
"""

gds.run_cypher(query_ingest, {"reports":reports})

### Chaining

In [ ]:
query_chaining = """
MATCH (x:Record)
WITH x ORDER BY x.record_number
WITH x.report_id AS report_id, collect(x) AS xs
CALL apoc.nodes.link(xs, 'NEXT');
"""

gds.run_cypher(query_chaining, {})

### Query

#### part 1

In [ ]:
gds.run_cypher('CREATE INDEX record_val IF NOT EXISTS FOR (r:Record) ON (r.value)')
gds.run_cypher('CREATE INDEX record_record_number IF NOT EXISTS FOR (r:Record) ON (r.record_number)');

In [ ]:
query_part1 = """
CYPHER runtime=parallel
MATCH (first:Record {record_number: 0})-[:NEXT]->(second)
WHERE first.value <> second.value
WITH first, second, sign(second.value-first.value) AS sign
MATCH path = (first)(
  (x)-[:NEXT]->(y)
  WHERE 1 <= sign * (y.value - x.value) <= 3
)*(last WHERE NOT EXISTS {(last)-[:NEXT]->()})
RETURN count(path) AS part1
"""

gds.run_cypher(query_part1, {})

#### part 2

In [ ]:
query_square_chaining = """
MATCH (x)-[:NEXT]->{2}(y)
MERGE (x)-[:NEXT_SQUARE]->(y)
"""

query_skip_first = """
MATCH (first:Record {record_number:0})-[:NEXT]->(second)
MERGE (first)-[:SKIP_FIRST]->(second)
"""

query_skip_last = """
MATCH (before_last:Record)-[:NEXT]->(last)
WHERE NOT EXISTS {(last)-[:NEXT]->()}
MERGE (before_last)-[:SKIP_LAST]->(last)
"""

for q in [query_square_chaining, query_skip_first, query_skip_last]:
    gds.run_cypher(q, {})

In [ ]:
query_part2 = """
CYPHER runtime=parallel
MATCH (:Record {record_number:0})-[rsf:SKIP_FIRST]->{0,1}(first)-[rf:NEXT|NEXT_SQUARE]->(second)
WHERE 1<= abs(second.value-first.value)<=3
WITH first, rsf, rf, second, sign(second.value-first.value) AS sign
MATCH path=(second)
(
  (x)-[rs:NEXT|NEXT_SQUARE]->(y)
  WHERE 1 <= sign * (y.value - x.value) <= 3
)*
()-[rsl:SKIP_LAST]->{0,1}(last WHERE NOT EXISTS {(last)-[:NEXT]->()})
WHERE size([r IN rsf+[rf]+rs+rsl WHERE r:NEXT_SQUARE OR r:SKIP_FIRST OR r:SKIP_LAST ]) <=1
RETURN count(DISTINCT first.report_id) AS part2
"""

gds.run_cypher(query_part2, {})